In [1]:
import numpy as np
import scipy as sp
import pandas as pd
import timeit
import datetime
import time
import pprint
import itertools
import pickle
import sklearn
import dask
import os
os.chdir('/mnt/t48/bighomes-active/sfeng/patentdiffusion/')
import fastparquet
seed = 3
import seaborn as sns
import matplotlib
import matplotlib.pyplot as plt
import h5py
import collections

/homedir/eco/sfeng/bigdata/python/miniconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [5]:
mdc = fastparquet.ParquetFile("DataStore/2018-07-P2/mutual_cited_0716.parq").to_pandas()
print(len(mdc))
print(mdc.columns)

mdc = mdc.loc[(mdc["common_pat_inv"] == False) & (mdc["lawyer_match"] == False)]
print(len(mdc))

mdc["common_npc_match"] = (mdc["num_common_npc"] >= 1)
mdc["common_cited_match"] = (mdc["num_common_cited"] >= 1)     
mdc["primclass_match"] = (mdc["tp_primclass"] == mdc["op_primclass"])

ml = mdc.loc[mdc["primclass_match"] == True]

2445169
Index(['tp', 'op', 'num_common_cited', 'op_cites_tp', 'sim_ldavecs',
       'sim_docvecs', 'tp_gyear', 'tp_inv_msa', 'tp_naics_name',
       'tp_primclass', 'op_gyear', 'op_inv_msa', 'op_naics_name',
       'op_primclass', 'year_group', 'num_common_npc', 'bin_sim_docvecs',
       'inv_msa_match', 'num_common_pat_inv', 'common_pat_inv',
       'lawyer_match'],
      dtype='object')
2338958


In [6]:
m2 = mdc
res = {}

c = "op_cites_tp"

npc_T = m2.loc[m2["common_npc_match"] == True].groupby("year_group")
npc_F = m2.loc[m2["common_npc_match"] == False].groupby("year_group")

npc_T_mean = npc_T.mean()[c]
npc_F_mean = npc_F.mean()[c]

vals = [sp.stats.ttest_ind(npc_T.get_group(n)[c], npc_F.get_group(n)[c], nan_policy="omit")
       for n in npc_T.groups.keys()]
t = [i[0] for i in vals]
p = [i[1] for i in vals]

r = pd.DataFrame({"$I(Common\,NPC\geq 1) = T$": npc_T_mean.tolist(),
                                 "$I(Common\,NPC\geq 1) = F$": npc_F_mean.tolist(),
                                  "$t$-value":t,
                                  "$p$-value":p,
                                 }, index=npc_T_mean.index.tolist()).T
res["dc"] = np.round(r,3)

npc_T = m2.loc[(m2["common_npc_match"] == True) & (m2["inv_msa_match"] == True)].groupby("year_group")
npc_F = m2.loc[(m2["common_npc_match"] == True) & (m2["inv_msa_match"] == False)].groupby("year_group")

npc_T_mean = npc_T.mean()[c]
npc_F_mean = npc_F.mean()[c]

vals = [sp.stats.ttest_ind(npc_T.get_group(n)[c], npc_F.get_group(n)[c], nan_policy="omit")
       for n in npc_T.groups.keys()]
t = [i[0] for i in vals]
p = [i[1] for i in vals]

r = pd.DataFrame({"$I(Common\,NPC\geq 1) = T \& I(MSA \,Match) = T$": npc_T_mean.tolist(),
                "$I(Common\,NPC\geq 1) = T \& I(MSA \,Match) = F$": npc_F_mean.tolist(),
                                  "$t$-value":t,
                                  "$p$-value":p,
                                 }, index=npc_T_mean.index.tolist()).T
res["dc_msa"] = np.round(r,3)

c = "sim_docvecs"

npc_T = m2.loc[m2["common_npc_match"] == True].groupby("year_group")
npc_F = m2.loc[m2["common_npc_match"] == False].groupby("year_group")

npc_T_mean = npc_T.mean()[c]
npc_F_mean = npc_F.mean()[c]

vals = [sp.stats.ttest_ind(npc_T.get_group(n)[c], npc_F.get_group(n)[c], nan_policy="omit")
       for n in npc_T.groups.keys()]
t = [i[0] for i in vals]
p = [i[1] for i in vals]

r = pd.DataFrame({"$I(Common\,NPC\geq 1) = T$": npc_T_mean.tolist(),
                                 "$I(Common\,NPC\geq 1) = F$": npc_F_mean.tolist(),
                                  "$t$-value":t,
                                  "$p$-value":p,
                                 }, index=npc_T_mean.index.tolist()).T
res["sim_docvecs"] = np.round(r,3)

npc_T = m2.loc[(m2["common_npc_match"] == True) & (m2["inv_msa_match"] == True)].groupby("year_group")
npc_F = m2.loc[(m2["common_npc_match"] == True) & (m2["inv_msa_match"] == False)].groupby("year_group")

npc_T_mean = npc_T.mean()[c]
npc_F_mean = npc_F.mean()[c]

vals = [sp.stats.ttest_ind(npc_T.get_group(n)[c], npc_F.get_group(n)[c], nan_policy="omit")
       for n in npc_T.groups.keys()]
t = [i[0] for i in vals]
p = [i[1] for i in vals]

r = pd.DataFrame({"$I(Common\,NPC\geq 1) = T \& I(MSA \,Match) = T$": npc_T_mean.tolist(),
                "$I(Common\,NPC\geq 1) = T \& I(MSA \,Match) = F$": npc_F_mean.tolist(),
                                  "$t$-value":t,
                                  "$p$-value":p,
                                 }, index=npc_T_mean.index.tolist()).T
res["sim_docvecs_msa"] = np.round(r,3)

/homedir/eco/sfeng/bigdata/python/miniconda3/lib/python3.6/site-packages/numpy/core/fromnumeric.py:3157: RuntimeWarning: Degrees of freedom <= 0 for slice
  **kwargs)
/homedir/eco/sfeng/bigdata/python/miniconda3/lib/python3.6/site-packages/numpy/core/_methods.py:132: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


In [7]:
for k,v in res.items():
    print(k)
    print(v.to_latex(index=True, escape=False, column_format="lcccc"))

dc
\begin{tabular}{lcccc}
\toprule
{} &  1975-85 &  1985-95 &  1995-05 &  2005-15 \\
\midrule
$I(Common\,NPC\geq 1) = F$ &    0.067 &    0.058 &    0.026 &    0.010 \\
$I(Common\,NPC\geq 1) = T$ &    0.000 &    0.091 &    0.042 &    0.032 \\
$p$-value                  &    0.591 &    0.216 &    0.075 &    0.000 \\
$t$-value                  &   -0.537 &    1.236 &    1.780 &   26.352 \\
\bottomrule
\end{tabular}

dc_msa
\begin{tabular}{lcccc}
\toprule
{} &  1975-85 &  1985-95 &  1995-05 &  2005-15 \\
\midrule
$I(Common\,NPC\geq 1) = T \& I(MSA \,Match) = F$ &      0.0 &    0.160 &    0.063 &    0.019 \\
$I(Common\,NPC\geq 1) = T \& I(MSA \,Match) = T$ &      0.0 &    0.058 &    0.026 &    0.058 \\
$p$-value                                        &      NaN &    0.147 &    0.100 &    0.000 \\
$t$-value                                        &      NaN &   -1.464 &   -1.650 &   12.718 \\
\bottomrule
\end{tabular}

sim_docvecs
\begin{tabular}{lcccc}
\toprule
{} &  1975-85 &  1985-95 &  19

In [8]:
m2 = ml
res = {}

c = "op_cites_tp"

npc_T = m2.loc[m2["common_npc_match"] == True].groupby("year_group")
npc_F = m2.loc[m2["common_npc_match"] == False].groupby("year_group")

npc_T_mean = npc_T.mean()[c]
npc_F_mean = npc_F.mean()[c]

vals = [sp.stats.ttest_ind(npc_T.get_group(n)[c], npc_F.get_group(n)[c], nan_policy="omit")
       for n in npc_T.groups.keys()]
t = [i[0] for i in vals]
p = [i[1] for i in vals]

r = pd.DataFrame({"$I(Common\,NPC\geq 1) = T$": npc_T_mean.tolist(),
                                 "$I(Common\,NPC\geq 1) = F$": npc_F_mean.tolist(),
                                  "$t$-value":t,
                                  "$p$-value":p,
                                 }, index=npc_T_mean.index.tolist()).T
res["dc"] = np.round(r,3)

npc_T = m2.loc[(m2["common_npc_match"] == True) & (m2["inv_msa_match"] == True)].groupby("year_group")
npc_F = m2.loc[(m2["common_npc_match"] == True) & (m2["inv_msa_match"] == False)].groupby("year_group")

npc_T_mean = npc_T.mean()[c]
npc_F_mean = npc_F.mean()[c]

vals = [sp.stats.ttest_ind(npc_T.get_group(n)[c], npc_F.get_group(n)[c], nan_policy="omit")
       for n in npc_T.groups.keys()]
t = [i[0] for i in vals]
p = [i[1] for i in vals]

r = pd.DataFrame({"$I(Common\,NPC\geq 1) = T \& I(MSA \,Match) = T$": npc_T_mean.tolist(),
                "$I(Common\,NPC\geq 1) = T \& I(MSA \,Match) = F$": npc_F_mean.tolist(),
                                  "$t$-value":t,
                                  "$p$-value":p,
                                 }, index=npc_T_mean.index.tolist()).T
res["dc_msa"] = np.round(r,3)

c = "sim_docvecs"

npc_T = m2.loc[m2["common_npc_match"] == True].groupby("year_group")
npc_F = m2.loc[m2["common_npc_match"] == False].groupby("year_group")

npc_T_mean = npc_T.mean()[c]
npc_F_mean = npc_F.mean()[c]

vals = [sp.stats.ttest_ind(npc_T.get_group(n)[c], npc_F.get_group(n)[c], nan_policy="omit")
       for n in npc_T.groups.keys()]
t = [i[0] for i in vals]
p = [i[1] for i in vals]

r = pd.DataFrame({"$I(Common\,NPC\geq 1) = T$": npc_T_mean.tolist(),
                                 "$I(Common\,NPC\geq 1) = F$": npc_F_mean.tolist(),
                                  "$t$-value":t,
                                  "$p$-value":p,
                                 }, index=npc_T_mean.index.tolist()).T
res["sim_docvecs"] = np.round(r,3)

npc_T = m2.loc[(m2["common_npc_match"] == True) & (m2["inv_msa_match"] == True)].groupby("year_group")
npc_F = m2.loc[(m2["common_npc_match"] == True) & (m2["inv_msa_match"] == False)].groupby("year_group")

npc_T_mean = npc_T.mean()[c]
npc_F_mean = npc_F.mean()[c]

vals = [sp.stats.ttest_ind(npc_T.get_group(n)[c], npc_F.get_group(n)[c], nan_policy="omit")
       for n in npc_T.groups.keys()]
t = [i[0] for i in vals]
p = [i[1] for i in vals]

r = pd.DataFrame({"$I(Common\,NPC\geq 1) = T \& I(MSA \,Match) = T$": npc_T_mean.tolist(),
                "$I(Common\,NPC\geq 1) = T \& I(MSA \,Match) = F$": npc_F_mean.tolist(),
                                  "$t$-value":t,
                                  "$p$-value":p,
                                 }, index=npc_T_mean.index.tolist()).T
res["sim_docvecs_msa"] = np.round(r,3)

/homedir/eco/sfeng/bigdata/python/miniconda3/lib/python3.6/site-packages/numpy/core/fromnumeric.py:3157: RuntimeWarning: Degrees of freedom <= 0 for slice
  **kwargs)
/homedir/eco/sfeng/bigdata/python/miniconda3/lib/python3.6/site-packages/numpy/core/_methods.py:132: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


In [12]:
for k,v in res.items():
    print(k)
    print(v.to_latex(index=True, escape=False, column_format="lcccc"))

dc
\begin{tabular}{lcccc}
\toprule
{} &  1975-85 &  1985-95 &  1995-05 &  2005-15 \\
\midrule
$I(Common\,NPC\geq 1) = F$ &    0.094 &    0.086 &    0.046 &    0.018 \\
$I(Common\,NPC\geq 1) = T$ &    0.000 &    0.029 &    0.045 &    0.034 \\
$p$-value                  &    0.578 &    0.240 &    0.993 &    0.000 \\
$t$-value                  &   -0.557 &   -1.176 &   -0.009 &   10.287 \\
\bottomrule
\end{tabular}

dc_msa
\begin{tabular}{lcccc}
\toprule
{} &  1975-85 &  1985-95 &  1995-05 &  2005-15 \\
\midrule
$I(Common\,NPC\geq 1) = T \& I(MSA \,Match) = F$ &      0.0 &    0.091 &    0.062 &    0.015 \\
$I(Common\,NPC\geq 1) = T \& I(MSA \,Match) = T$ &      0.0 &    0.000 &    0.032 &    0.084 \\
$p$-value                                        &      NaN &    0.151 &    0.341 &    0.000 \\
$t$-value                                        &      NaN &   -1.471 &   -0.954 &   15.336 \\
\bottomrule
\end{tabular}

sim_docvecs
\begin{tabular}{lcccc}
\toprule
{} &  1975-85 &  1985-95 &  19

In [13]:
print(0.382/0.258, 0.548/0.25)

1.4806201550387597 2.192


____________
### Previous

In [82]:
res["dc"]

,"$I(Common\,NPC\geq 1) = F$","$I(Common\,NPC\geq 1) = T$",$p$-value,$t$-value
1975-85,0.094,0.000,0.578,-0.557
1985-95,0.086,0.029,0.240,-1.176
1995-05,0.046,0.045,0.993,-0.009
2005-15,0.018,0.034,0.000,10.287


In [16]:
display(mdc[["num_common_npc", "inv_msa_match"]].groupby("inv_msa_match").mean())
display(m2[["num_common_npc", "inv_msa_match"]].groupby("inv_msa_match").mean())
display(mdc[["common_npc_match", "inv_msa_match"]].groupby("inv_msa_match").mean())
display(mdc[["common_npc_match", "inv_msa_match"]].groupby("common_npc_match").mean())
display(m2[["common_npc_match", "inv_msa_match"]].groupby("inv_msa_match").mean())
display(m2[["common_npc_match", "inv_msa_match"]].groupby("common_npc_match").mean())
display(mdc[["num_common_npc", "inv_msa_match", "op_cites_tp"]]\
        .groupby(["num_common_npc", "inv_msa_match"]).mean())
display(m2[["num_common_npc", "inv_msa_match", "op_cites_tp"]]\
        .groupby(["num_common_npc", "inv_msa_match"]).mean())

,num_common_npc
inv_msa_match,
False,0.060515
True,0.580553


,num_common_npc
inv_msa_match,
False,0.119382
True,0.749326


,common_npc_match
inv_msa_match,
False,0.004412
True,0.025237


,inv_msa_match
common_npc_match,
False,0.083381
True,0.347026


,common_npc_match
inv_msa_match,
False,0.009685
True,0.034103


,inv_msa_match
common_npc_match,
False,0.096883
True,0.279176


op_cites_tp
num_common_npc inv_msa_match             
0              False             0.018927
               True              0.021732
1              False             0.042506
               True              0.044420
2              False             0.027888
               True              0.027726
3              False             0.010684
               True              0.028278
4              False             0.015909
               True              0.019231
5              False             0.012545
               True              0.068063
6              False             0.005042
               True              0.076389
7              False             0.004757
               True              0.061538
8              False             0.000000
               True              0.033333
9              False             0.003556
               True              0.041322
10             False             0.009804
               True              0.049383
11             False             0.025000
               True              0.062500
12             False             0.021739
               True              0.112903
13             False             0.009901
               True              0.081081
14             False             0.060241
               True              0.147541
...                                   ...
373            False             0.000000
376            True              0.000000
383            True              0.000000
386            True              0.000000
390            False             0.000000
401            True              0.500000
405            True              0.000000
432            True              0.000000
478            False             0.000000
484            False             0.000000
494            False             0.000000
               True              0.000000
496            False             0.000000
580            False             0.000000
583            True              0.000000
585            True              0.000000
602            True              0.000000
606            True              0.000000
609            True              0.000000
611            True              0.000000
618            False             0.000000
637            False             0.000000
638            False             0.000000
642            False             0.000000
655            True              0.000000
674            True              0.000000
693            True              0.000000
724            True              0.000000
733            True              0.000000
741            True              0.000000

[419 rows x 1 columns]

op_cites_tp
num_common_npc inv_msa_match             
0              False             0.035803
               True              0.034648
1              False             0.036554
               True              0.061538
2              False             0.028226
               True              0.021552
3              False             0.004000
               True              0.036649
4              False             0.009259
               True              0.016529
5              False             0.003096
               True              0.080000
6              False             0.002571
               True              0.067797
7              False             0.000000
               True              0.095238
8              False             0.000000
               True              0.024096
9              False             0.001175
               True              0.036364
10             False             0.022222
               True              0.071429
11             False             0.047619
               True              0.103448
12             False             0.016129
               True              0.200000
13             False             0.000000
               True              0.125000
14             False             0.081633
               True              0.230769
...                                   ...
213            False             0.000000
217            True              0.000000
218            True              0.000000
221            False             0.000000
               True              0.000000
224            True              0.000000
225            False             0.000000
227            True              1.000000
228            False             0.000000
               True              0.000000
230            False             0.000000
233            True              0.000000
237            False             0.000000
239            False             0.000000
240            False             0.000000
251            False             0.000000
252            False             0.000000
255            False             0.000000
257            False             0.000000
273            False             0.000000
290            False             0.000000
300            False             0.000000
320            False             0.000000
331            False             0.000000
356            False             0.000000
390            False             0.000000
401            True              0.000000
432            True              0.000000
606            True              0.000000
637            False             0.000000

[301 rows x 1 columns]

In [27]:
sp.stats.ttest_ind(m2.loc[(m2["common_npc_match"] == True) & (m2["inv_msa_match"] == False),
                   "sim_docvecs"].values,
             m2.loc[(m2["common_npc_match"] == True) & (m2["inv_msa_match"] == True),
                   "sim_docvecs"].values
            )

Ttest_indResult(statistic=45.097253780703674, pvalue=0.0)

In [34]:
m2[["common_npc_match", "sim_docvecs", "year_group", "inv_msa_match"]].groupby(["common_npc_match", "year_group", "inv_msa_match"]).mean()

sim_docvecs
common_npc_match year_group inv_msa_match             
False            1975-85    False             0.290912
                            True              0.317070
                 1985-95    False             0.278321
                            True              0.304164
                 1995-05    False             0.256239
                            True              0.278059
                 2005-15    False             0.247982
                            True              0.267487
True             1975-85    False             0.267720
                            True              0.742468
                 1985-95    False             0.305575
                            True              0.446162
                 1995-05    False             0.368260
                            True              0.393211
                 2005-15    False             0.610285
                            True              0.381180

In [35]:
m2[["common_npc_match", "op_cites_tp", "year_group", "inv_msa_match"]].groupby(["common_npc_match", "year_group", "inv_msa_match"]).mean()

op_cites_tp
common_npc_match year_group inv_msa_match             
False            1975-85    False             0.093462
                            True              0.097166
                 1985-95    False             0.085818
                            True              0.087292
                 1995-05    False             0.045505
                            True              0.046370
                 2005-15    False             0.018037
                            True              0.018501
True             1975-85    False             0.000000
                            True              0.000000
                 1985-95    False             0.090909
                            True              0.000000
                 1995-05    False             0.061728
                            True              0.031579
                 2005-15    False             0.015040
                            True              0.084428

In [31]:
m2[["common_npc_match", "op_cites_tp", "year_group"]].groupby(["common_npc_match", "year_group"]).mean()

op_cites_tp
common_npc_match year_group             
False            1975-85        0.093694
                 1985-95        0.085921
                 1995-05        0.045591
                 2005-15        0.018084
True             1975-85        0.000000
                 1985-95        0.029412
                 1995-05        0.045455
                 2005-15        0.033885

In [32]:
m2[["common_npc_match", "sim_docvecs", "year_group"]].groupby(["common_npc_match", "year_group"]).mean()

sim_docvecs
common_npc_match year_group             
False            1975-85        0.292555
                 1985-95        0.280120
                 1995-05        0.258413
                 2005-15        0.249937
True             1975-85        0.425969
                 1985-95        0.400678
                 1995-05        0.381728
                 2005-15        0.548062

In [28]:
sp.stats.ttest_ind(m2.loc[(m2["common_npc_match"] == True) & (m2["inv_msa_match"] == False),
                   "op_cites_tp"].values,
             m2.loc[(m2["common_npc_match"] == True) & (m2["inv_msa_match"] == True),
                   "op_cites_tp"].values
            )

Ttest_indResult(statistic=-14.721600833948761, pvalue=1.985745717748611e-48)

In [23]:
display(m2[["common_npc_match", "inv_msa_match", "sim_docvecs"]]\
        .groupby(["common_npc_match", "inv_msa_match"]).mean())
display(m2[["common_npc_match", "inv_msa_match", "op_cites_tp"]]\
        .groupby(["common_npc_match", "inv_msa_match"]).mean())
display(m2[["inv_msa_match", "op_cites_tp"]]\
        .groupby(["inv_msa_match"]).mean())
display(m2[["inv_msa_match", "sim_docvecs"]]\
        .groupby(["inv_msa_match"]).mean())

sim_docvecs
common_npc_match inv_msa_match             
False            False             0.254474
                 True              0.274433
True             False             0.606218
                 True              0.382512

op_cites_tp
common_npc_match inv_msa_match             
False            False             0.035803
                 True              0.034648
True             False             0.015829
                 True              0.081297

,op_cites_tp
inv_msa_match,
False,0.035609
True,0.036239


,sim_docvecs
inv_msa_match,
False,0.257881
True,0.278119


In [93]:
display(mdc[["inv_msa_match", "sim_docvecs", "op_cites_tp"]].groupby(["inv_msa_match"]).mean())
display(m2[["inv_msa_match", "sim_docvecs", "op_cites_tp"]].groupby(["inv_msa_match"]).mean())


,sim_docvecs,op_cites_tp
inv_msa_match,,
False,0.216230,0.018930
True,0.238106,0.022621


,sim_docvecs,op_cites_tp
inv_msa_match,,
False,0.257881,0.035609
True,0.278119,0.036239
